# PyTorch with C++. Linear regression model

> 'Develop a linear regression model with PyTorch C++ frontend'

- toc:true
- branch: master
- badges: false
- comments: false
- author: Alexandros Giavaras
- categories: [PyTorch, C++, API, machine-learning, linear-regression]

## Overview

## Linear regression model

```
#include <torch/torch.h>

#include <iostream>
#include <vector>

namespace example
{

typedef std::size_t uint_t;

class Net: public torch::nn::Module
{

public:

	//
	Net(uint_t input_size, uint_t output_size);
	
	// forward
	torch::Tensor forward(torch::Tensor input);
	
private:

	torch::nn::Linear linear;
	torch::Tensor bias_;

};

Net::Net(uint_t input_size, uint_t output_size)
:
linear(register_module("linear", torch::nn::Linear(input_size, output_size))),
bias_()
{
	bias_ = register_parameter("b", torch::randn(output_size));
}

torch::Tensor 
Net::forward(torch::Tensor input){
	return linear(input) + bias_;
}

}

int main() {


  using namespace  example;
  
  if(torch::cuda::is_available()){
  	std::cout<<"CUDA is available on this machine"<<std::endl;
  }
  else{
  	std::cout<<"CUDA is not available on this machine"<<std::endl;
  }
  
  // create data
  std::vector<double> x_train(11, 0.0);
  std::vector<double> y_train(11, 0.0);
  
  for(uint_t i=0; i<x_train.size(); ++i){
  	x_train[i] = static_cast<double>(i);
  	y_train[i] = 2*static_cast<double>(i) + 1;
  }
                  
  auto x_tensor = torch::from_blob(x_train.data(), {int(y_train.size()), int(x_train.size()/y_train.size())});
  auto y_tensor = torch::from_blob(y_train.data(), {int(y_train.size()), 1});
  
  Net net(1, 1);
  for (const auto& p : net.parameters()) {
    std::cout << p << std::endl;
  }
  
  torch::nn::MSELoss mse;
  torch::optim::SGD sgd(net.parameters(), 0.01);
  
  for(uint_t e=0; e<100; ++e){
  
  	sgd.zero_grad();
  	
  	auto outputs = net.forward(x_tensor);
  	auto loss = mse(outputs, y_tensor);
  	
  	// get gradients w.r.t to parameters
    	loss.backward();

    	// update parameters
    	sgd.step();
    	std::cout<<"Epoch="<<e<<" loss="<<loss<<std::endl;
  }
  
  return 0;
}
```

```
CUDA is not available on this machine
0.01 *
 1.7510
[ CPUFloatType{1} ]
-0.4187
[ CPUFloatType{1,1} ]
0.01 *
-1.6082
[ CPUFloatType{1} ]
Epoch=0 loss=4.09091
[ CPUFloatType{} ]
Epoch=1 loss=3.72361
[ CPUFloatType{} ]
Epoch=2 loss=3.39557
[ CPUFloatType{} ]
Epoch=3 loss=3.10247
[ CPUFloatType{} ]
Epoch=4 loss=2.84047
[ CPUFloatType{} ]
Epoch=5 loss=2.60615
[ CPUFloatType{} ]
Epoch=6 loss=2.39647
[ CPUFloatType{} ]
Epoch=7 loss=2.20874
[ CPUFloatType{} ]
Epoch=8 loss=2.04054
[ CPUFloatType{} ]
Epoch=9 loss=1.88976
[ CPUFloatType{} ]
Epoch=10 loss=1.75448
[ CPUFloatType{} ]
Epoch=11 loss=1.63302
[ CPUFloatType{} ]
Epoch=12 loss=1.52387
[ CPUFloatType{} ]
Epoch=13 loss=1.42571
[ CPUFloatType{} ]
...

```

## References